<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_05/class_005_feature_extraction_image_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
try:
  import glrlm
except:
  !pip install pip glrlm

In [4]:
import glrlm
from glrlm import GLRLM

In [5]:
# check https://pypi.org/project/glrlm/

In [6]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 318MB/s]


In [7]:
FILE='trash_nov_22_2018.zip'

In [8]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [9]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [10]:
Img_Size = 200
ref = 'jpg'
notref = 'met'

In [11]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [12]:
app = GLRLM()
img = X[0]
glrlm = app.get_features(img, 8)

In [13]:
glrlm.Features

[1.263, 3951.709, 11334.174, 7848.195, 15.972]

In [14]:
glrlm_names =['SRE','LRE','GLU','RLU','RPC']

In [15]:
'''
SRE = Short Run Emphasis
LRE = Long Run Emphasis
GLU = Grey Level Uniformity
RLU = Run Length Uniformity
RPC = Run Percentage
'''

'\nSRE = Short Run Emphasis\nLRE = Long Run Emphasis\nGLU = Grey Level Uniformity\nRLU = Run Length Uniformity\nRPC = Run Percentage\n'

In [16]:
app = GLRLM()

def glrlm_features(img,Img_Size,model=app):
  resized = cv2.resize(img, (Img_Size,Img_Size))
  glrlm_get = model.get_features(resized, 8)
  return glrlm_get.Features

In [17]:
def properties(glcm_names,img,Img_Size):
  glcm =[]
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# see https://github.com/ucfilho/ANN/blob/master/ANN_2022/Image/class_001_image_processing_smart_trash_bin_012.ipynb
features =[]
# for i in range(len(img_name)):
for i in range(len(img_name)):
  glcm = []
  glrlm_add = []
  img = X[i]
  glcm = properties(glcm_names,img,Img_Size)
  glrlm_add = glrlm_features(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  values_green = properties(glcm_names,green_img,Img_Size)
  glrlm_green = glrlm_features(green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties(glcm_names,red_img,Img_Size)
  glrlm_red = glrlm_features(red_img,Img_Size)

  blue_img = img[:,:,2]
  values_blue = properties(glcm_names,blue_img,Img_Size)
  glrlm_blue = glrlm_features(blue_img,Img_Size)

  total= [*glcm,*values_red, *values_green, *values_blue, *glrlm_add, *glrlm_red, *glrlm_green, *glrlm_blue]

  features.append(total)



In [18]:
glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
glrlm_names =['SRE','LRE','GLU','RLU','RPC']

def new_names(letter,names):
  lista = []
  for x in names:
    lista.append(x+letter)
  return lista

names = glcm_names + new_names('_r',glcm_names)
names = names + new_names('_g',glcm_names)
names = names + new_names('_b',glcm_names)


names = names + glrlm_names
names = names + new_names('_r',glrlm_names)
names = names + new_names('_g',glrlm_names)
names = names + new_names('_b',glrlm_names)


print(names)

df = pd.DataFrame(features,columns=names)

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM', 'contrast_r', 'dissimilarity_r', 'homogeneity_r', 'energy_r', 'correlation_r', 'ASM_r', 'contrast_g', 'dissimilarity_g', 'homogeneity_g', 'energy_g', 'correlation_g', 'ASM_g', 'contrast_b', 'dissimilarity_b', 'homogeneity_b', 'energy_b', 'correlation_b', 'ASM_b', 'SRE', 'LRE', 'GLU', 'RLU', 'RPC', 'SRE_r', 'LRE_r', 'GLU_r', 'RLU_r', 'RPC_r', 'SRE_g', 'LRE_g', 'GLU_g', 'RLU_g', 'RPC_g', 'SRE_b', 'LRE_b', 'GLU_b', 'RLU_b', 'RPC_b']


In [19]:
glcm_names

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

In [20]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [21]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [22]:
df['target']=label

In [23]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [24]:
y=df['target']
X=df.drop(['target'], axis='columns')

In [25]:
y = pd.get_dummies(y)
# check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
y

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
2522,0,0,0,0,0,1
2523,0,0,0,0,0,1
2524,0,0,0,0,0,1
2525,0,0,0,0,0,1


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=1)

In [27]:
scaler = StandardScaler()
# transform data
scaler_X = scaler.fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

In [28]:
nin = X_train.shape[1]
nout =y.shape[1]
nhidden = 30

In [29]:
import tensorflow as tf

In [30]:
# Create a `Sequential` model and add a Dense layer as the first layer.
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(nin,)))
model.add(tf.keras.layers.Dense(nhidden, activation='relu'))
# Now the model will take as input arrays of shape (None, 16)
# and output arrays of shape (None, 32).
# Note that after the first layer, you don't need to specify
# the size of the input anymore:
model.add(tf.keras.layers.Dense(nout, activation='softmax'))


In [31]:
X_test = np.array(X_test )
y_test = np.array(y_test)
y_test = y_test.astype(np.int32)
y_train = y_train.astype(np.int32)
X_train = X_train.reshape(-1, nin) 
X_test = X_test.reshape(-1, nin)

In [32]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [33]:
# gives us back a <keras.callbacks.History object at 0x112e61a90>
#r = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=15, batch_size=32)
r = model.fit(X_train, y_train, epochs=1000)
print("Returned:", r)

Epoch 1/1000
53/53 [==============================] - 1s 2ms/step - loss: 1.7014 - accuracy: 0.2741
Epoch 2/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.4814 - accuracy: 0.4170
Epoch 3/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3929 - accuracy: 0.4542
Epoch 4/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3402 - accuracy: 0.4749
Epoch 5/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2989 - accuracy: 0.4956
Epoch 6/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2707 - accuracy: 0.5097
Epoch 7/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2433 - accuracy: 0.5298
Epoch 8/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2263 - accuracy: 0.5286
Epoch 9/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2012 - accuracy: 0.5375
Epoch 10/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.1820 - accuracy: 0.5446

In [34]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=model.predict(X_train)
y_calc_test=model.predict(X_test)

In [35]:
y_calc_test

array([[1.4401767e-02, 2.6303553e-04, 5.5534241e-05, 9.8385423e-01,
        1.3355440e-03, 8.9881047e-05],
       [3.1540599e-03, 2.4381736e-02, 1.0536373e-04, 9.1063923e-01,
        2.5586996e-03, 5.9161022e-02],
       [2.4348225e-05, 3.7349791e-03, 5.2155005e-03, 8.6952168e-03,
        9.8192066e-01, 4.0939625e-04],
       ...,
       [1.5727710e-02, 4.1254508e-01, 4.7181228e-01, 4.2911290e-04,
        4.4989526e-02, 5.4496374e-02],
       [1.9262677e-06, 1.7310741e-08, 6.5033397e-05, 9.9955386e-01,
        3.5328773e-04, 2.5809653e-05],
       [5.8228128e-02, 6.5681005e-01, 7.8450748e-06, 2.5071266e-01,
        3.1885114e-02, 2.3562345e-03]], dtype=float32)

In [36]:
def retorna(y):
  resp = []
  for yi in y:
    vmax =max(yi)
    for idx, x in enumerate(yi):
      if(x == vmax):
        resp.append(idx)
  return resp

In [37]:
y_test[0]

array([0, 0, 0, 0, 1, 0], dtype=int32)

In [38]:
pd.__version__

'1.3.5'

In [39]:

y_calc = retorna(y_calc_test)
y_test = retorna(y_test)

In [40]:
confusion_matrix(y_test, y_calc)

array([[105,   5,   4,  13,   2,   4],
       [  2,  97,  27,  10,  23,   7],
       [  2,  26,  69,  12,  16,  10],
       [  8,  12,   6, 150,  18,   2],
       [  5,  15,  13,  22,  96,   8],
       [  2,   5,   6,   3,   9,  20]])

In [41]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       0.85      0.79      0.82       133
       glass       0.61      0.58      0.60       166
       metal       0.55      0.51      0.53       135
       paper       0.71      0.77      0.74       196
     plastic       0.59      0.60      0.59       159
       trash       0.39      0.44      0.42        45

    accuracy                           0.64       834
   macro avg       0.62      0.62      0.62       834
weighted avg       0.65      0.64      0.64       834



In [1]:
Arq = "output_excel.xlsx"
df.to_excel(Arq)
from google.colab import files
files.download(Arq)

NameError: ignored